In [1]:
# import packages
import pandas as pd

%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys
sys.path.append('../../../open-grid-emissions/src/')

from column_checks import get_dtypes
import load_data
from filepaths import *

year = 2021
path_prefix = f"{year}/"

In [3]:
pudl_out = load_data.initialize_pudl_out(year)
gens_eia860 = pudl_out.gens_eia860()
plants_eia860 = pudl_out.plants_eia860()

2023-08-16 15:34:31 [    INFO] catalystcoop.pudl.transform.eia861:456 Started with 323 missing BA Codes out of 13488 records (2.39%)
2023-08-16 15:34:33 [    INFO] catalystcoop.pudl.transform.eia861:480 Ended with 323 missing BA Codes out of 13488 records (2.39%)
2023-08-16 15:34:33 [    INFO] catalystcoop.pudl.output.eia860:177 97.6% of plant records have consistently reported BA Codes
2023-08-16 15:34:33 [    INFO] catalystcoop.pudl.output.eia860:227 Before any filling treatment has been applied. 2.4% of records have no BA codes
2023-08-16 15:34:33 [    INFO] catalystcoop.pudl.output.eia860:227 Backfilling and consistent value is the same. Filled w/ most consistent BA code. 2.4% of records have no BA codes
2023-08-16 15:34:33 [    INFO] catalystcoop.pudl.output.eia860:227 SWPP is most consistent value. Filled w/ oldest BA code. 2.4% of records have no BA codes
2023-08-16 15:34:33 [    INFO] catalystcoop.pudl.output.eia860:227 NWMT is most consistent value. Filled w/ oldest BA code. 2

In [25]:
eia_gens = gens_eia860.merge(
    plants_eia860, how="left", on="plant_id_eia", suffixes=(None, "_remove")
)

eia_gens = eia_gens[[col for col in eia_gens.columns if "_remove" not in col]]
eia_gens = eia_gens[
    [
        "plant_id_eia",
        "plant_name_eia",
        "generator_id",
        "rto_iso_lmp_node_id",
        "utility_name_eia",
        "transmission_distribution_owner_name",
        "transmission_distribution_owner_state",
        "balancing_authority_code_eia",
        "balancing_authority_name_eia",
        "iso_rto_code",
        "nerc_region",
        "latitude",
        "longitude",
        "street_address",
        "city",
        "county",
        "state",
        "zip_code",
        "capacity_mw",
        "grid_voltage_1_kv",
        "prime_mover_code",
        "energy_source_code_1",
        "energy_source_code_2",
        "multiple_fuels",
        "cofire_fuels",
        "switch_oil_gas",
        "energy_storage",
        "operating_date",
        "current_planned_operating_date",
        "operational_status",
        "retirement_date",
    ]
]

# to filter this a bit, get rid of generators in WECC, Texas, Alaska, and Hawaii
eia_gens = eia_gens[~eia_gens["nerc_region"].isin(["UNK","TRE","WECC"])]

eia_gens.to_csv("../../../carbon-flow-analysis/MISO/data/eia860_export.csv", index=False)

In [47]:
miso_gens = gens_eia860.merge(
    plants_eia860[["plant_id_eia", "balancing_authority_code_eia", "transmission_distribution_owner_name"]],
    how="left",
    on="plant_id_eia",
)

miso_gens = miso_gens[(miso_gens["balancing_authority_code_eia"] == "MISO") & (miso_gens["operational_status"] == "existing")]

In [48]:
miso_gens["capacity_mw"].sum()

198755.0

In [ ]:
transmission_distribution_owner_name

In [49]:
miso_storage = miso_gens[miso_gens["prime_mover_code"].isin(["BA","PS"])]

In [58]:
miso_storage["capacity_mw"].sum()

2473.9

In [51]:
miso_storage[miso_storage["transmission_distribution_owner_name"] == "ITC Transmission"]

,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,associated_combined_heat_power,bga_source,bypass_heat_recovery,capacity_mw,carbon_capture,city,cofire_fuels,county,current_planned_operating_date,data_maturity,deliver_power_transgrid,distributed_generation,duct_burners,energy_source_1_transport_1,energy_source_1_transport_2,energy_source_1_transport_3,energy_source_2_transport_1,energy_source_2_transport_2,energy_source_2_transport_3,energy_source_code_1,energy_source_code_2,energy_source_code_3,energy_source_code_4,energy_source_code_5,energy_source_code_6,energy_storage_capacity_mwh,ferc_qualifying_facility,fluidized_bed_tech,fuel_type_code_pudl,fuel_type_count,latitude,longitude,minimum_load_mw,multiple_fuels,nameplate_power_factor,net_capacity_mwdc,operating_date,operating_switch,operational_status,operational_status_code,original_planned_operating_date,other_combustion_tech,other_modifications_date,other_planned_modifications,owned_by_non_utility,ownership_code,planned_derate_date,planned_energy_source_code_1,planned_modifications,planned_net_summer_capacity_derate_mw,planned_net_summer_capacity_uprate_mw,planned_net_winter_capacity_derate_mw,planned_net_winter_capacity_uprate_mw,planned_new_capacity_mw,planned_new_prime_mover_code,planned_repower_date,planned_retirement_date,planned_uprate_date,previously_canceled,prime_mover_code,pulverized_coal_tech,reactive_power_output_mvar,retirement_date,rto_iso_lmp_node_id,rto_iso_location_wholesale_reporting_id,solid_fuel_gasification,startup_source_code_1,startup_source_code_2,startup_source_code_3,startup_source_code_4,state,stoker_tech,street_address,subcritical_tech,summer_capacity_estimate,summer_capacity_mw,summer_estimated_capability_mw,supercritical_tech,switch_oil_gas,syncronized_transmission_grid,technology_description,time_cold_shutdown_full_load_code,timezone,topping_bottoming_code,turbines_inverters_hydrokinetics,turbines_num,ultrasupercritical_tech,unit_id_pudl,uprate_derate_completed_date,uprate_derate_during_year,winter_capacity_estimate,winter_capacity_mw,winter_estimated_capability_mw,zip_code,balancing_authority_code_eia,transmission_distribution_owner_name
3803,2021-01-01,1713,880,Ludington,4254,81,Consumers Energy Co,1,False,<NA>,False,329.8,<NA>,Ludington,<NA>,Mason,NaT,final,<NA>,<NA>,False,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,WAT,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,hydro,1,43.8942,-86.4447,240.0,<NA>,0.85,NaN,1973-01-01,<NA>,existing,OP,NaT,<NA>,NaT,<NA>,<NA>,J,NaT,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,NaT,NaT,NaT,<NA>,PS,<NA>,NaN,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,MI,<NA>,3525 S. Lakeshore Dr.,<NA>,<NA>,362.7,NaN,<NA>,<NA>,False,Hydroelectric Pumped Storage,10M,America/Detroit,X,<NA>,<NA>,<NA>,<NA>,NaT,False,<NA>,361.0,NaN,49431,MISO,ITC Transmission
3804,2021-01-01,1713,880,Ludington,4254,81,Consumers Energy Co,2,False,<NA>,False,329.8,<NA>,Ludington,<NA>,Mason,NaT,final,<NA>,<NA>,False,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,WAT,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,hydro,1,43.8942,-86.4447,240.0,<NA>,0.85,NaN,1973-03-01,<NA>,existing,OP,NaT,<NA>,NaT,<NA>,<NA>,J,NaT,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,NaT,NaT,NaT,<NA>,PS,<NA>,NaN,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,MI,<NA>,3525 S. Lakeshore Dr.,<NA>,<NA>,362.6,NaN,<NA>,<NA>,False,Hydroelectric Pumped Storage,10M,America/Detroit,X,<NA>,<NA>,<NA>,<NA>,NaT,False,<NA>,361.0,NaN,49431,MISO,ITC Transmission
3805,2021-01-01,1713,880,Ludington,4254,81,Consumers Energy Co,3,False,<NA>,False,329.8,<NA>,Ludington,<NA>,Mason,NaT,final,<NA>,<NA>,False,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,WAT,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,hydro,1,43.8942,-86.4447,240.0,<NA>,0.85,NaN,1973-04-01,<NA>,existing,OA,NaT,<NA>,NaT,<NA>,<NA>,J,NaT,<NA>,<NA>,NaN,49.0,NaN,32.0,NaN,<NA>,NaT,NaT,2022-05-01,<NA>,PS,<NA>,NaN,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,MI,<NA>,3525 S. Lakeshore Dr.,<NA>,<NA>,331.0,NaN,<NA>,<NA>,False,Hydroelectric Pumped Storage,10M,America/Detroit,X,<NA>,<NA>,<NA

In [50]:
miso_storage.groupby("transmission_distribution_owner_name")["capacity_mw"].sum()

transmission_distribution_owner_name
Ameren Illinois Company              0.3
Connexus Energy                     15.0
Consumers Energy Co                  1.4
Duke Energy Indiana, LLC            15.0
Entergy Arkansas Inc                30.0
Entergy New Orleans, LLC             0.5
ITC Midwest LLC                      0.3
ITC Transmission                  1978.8
Indianapolis Power & Light Co       20.0
Interstate Power and Light Co        2.5
MidAmerican Energy Co                1.1
Southern Indiana Gas & Elec Co       1.0
Union Electric Co - (MO)           408.0
Name: capacity_mw, dtype: float64

In [57]:
miso_gens[miso_gens["transmission_distribution_owner_name"] == "ITC Transmission"].capacity_mw.sum()

21095.8

2023-08-12 16:34:30 [    INFO] catalystcoop.pudl.transform.eia861:456 Started with 323 missing BA Codes out of 13488 records (2.39%)
2023-08-12 16:34:34 [    INFO] catalystcoop.pudl.transform.eia861:480 Ended with 323 missing BA Codes out of 13488 records (2.39%)
2023-08-12 16:34:34 [    INFO] catalystcoop.pudl.output.eia860:177 97.6% of plant records have consistently reported BA Codes
2023-08-12 16:34:34 [    INFO] catalystcoop.pudl.output.eia860:227 Before any filling treatment has been applied. 2.4% of records have no BA codes
2023-08-12 16:34:34 [    INFO] catalystcoop.pudl.output.eia860:227 Backfilling and consistent value is the same. Filled w/ most consistent BA code. 2.4% of records have no BA codes
2023-08-12 16:34:34 [    INFO] catalystcoop.pudl.output.eia860:227 SWPP is most consistent value. Filled w/ oldest BA code. 2.4% of records have no BA codes
2023-08-12 16:34:34 [    INFO] catalystcoop.pudl.output.eia860:227 NWMT is most consistent value. Filled w/ oldest BA code. 2

In [ ]:
multi = gens_eia860[["plant_id_eia","generator_id","multiple_fuels","cofire_fuels", "switch_oil_gas"]].merge(plants_eia860[["plant_id_eia","balancing_authority_code_eia"]], how="left", on="plant_id_eia")
multi = multi[multi["balancing_authority_code_eia"] == "MISO"]

In [ ]:
multi.cofire_fuels.sum()

In [ ]:
multi.multiple_fuels.sum()

In [ ]:
multi.switch_oil_gas.sum()

In [ ]:
multi.groupby(["multiple_fuels","cofire_fuels"], dropna=False).count()

In [ ]:
multi.cofire_fuels.sum()

In [ ]:
gens_eia860.filter(like="oil")

In [ ]:

plant_frequency = pudl_out.plants_eia860()[
    ["plant_id_eia", "plant_name_eia", "state", "reporting_frequency_code"]
]
plant_data = plant_data.merge(
    plant_frequency, how="left", on="plant_id_eia", validate="1:1"
)

plant_data

# How much of the data reported in EIA-923 comes from annually-reported plants?

In [ ]:
# load annual plant level outputs from OGE
plant_data = pd.read_csv(
    results_folder(f"{year}/plant_data/annual/us_units/plant_data.csv")
)[
    [
        "plant_id_eia",
        "net_generation_mwh",
        "fuel_consumed_for_electricity_mmbtu",
        "co2_mass_lb_for_electricity",
    ]
]

# merge plant attributes
plant_attributes = pd.read_csv(
    outputs_folder(f"{path_prefix}/plant_static_attributes_{year}.csv"),
    dtype=get_dtypes(),
)
plant_data = plant_data.merge(
    plant_attributes[
        [
            "plant_id_eia",
            "ba_code",
            "plant_primary_fuel",
            "fuel_category",
            "data_availability",
        ]
    ],
    how="outer",
    on="plant_id_eia",
    validate="1:1",
)

# merge reporting frequency
pudl_out = load_data.initialize_pudl_out(year)
plant_frequency = pudl_out.plants_eia860()[
    ["plant_id_eia", "plant_name_eia", "state", "utility_name_eia", "transmission_distribution_owner_name", "reporting_frequency_code"]
]
plant_data = plant_data.merge(
    plant_frequency, how="left", on="plant_id_eia", validate="1:1"
)

plant_data

In [ ]:
# filter to MISO only plants
miso_data = plant_data[plant_data["ba_code"] == "MISO"].copy()

# add a column for clean vs emitting
miso_data["emissions_category"] = "emitting"
miso_data.loc[
    miso_data["fuel_category"].isin(["hydro", "nuclear", "solar", "wind"]),
    "emissions_category",
] = "non-emitting"
miso_data.loc[
    miso_data["plant_primary_fuel"].isin(["MWH","PUR", "WH"]),
    "emissions_category",
] = "non-emitting"

# change AM frequency to A
miso_data.loc[
    miso_data["reporting_frequency_code"] == "AM", "reporting_frequency_code"
] = "A"

miso_data.loc[
    miso_data["data_availability"] == "cems_only", "data_availability"
] = "cems_and_eia"
miso_data["data_availability"] = miso_data[
    "data_availability"
].cat.remove_unused_categories()


# add reporting lag column
miso_data["reporting_lag"] = "no_data_reported"
miso_data.loc[
    (miso_data["data_availability"] == "eia_only")
    & (miso_data["reporting_frequency_code"] == "A"),
    "reporting_lag",
] = "10-22 months"
miso_data.loc[
    (miso_data["data_availability"] == "eia_only")
    & (miso_data["reporting_frequency_code"] == "M"),
    "reporting_lag",
] = "2-3 months"
miso_data.loc[
    (miso_data["data_availability"] == "eia_only")
    & (miso_data["reporting_frequency_code"].isna()),
    "reporting_lag",
] = "10-22 months"  # assume worst case
miso_data.loc[
    (miso_data["data_availability"] == "cems_and_eia")
    & (miso_data["reporting_frequency_code"] == "M"),
    "reporting_lag",
] = "1-3 months"
miso_data.loc[
    (miso_data["data_availability"] == "cems_and_eia")
    & (miso_data["reporting_frequency_code"] == "A"),
    "reporting_lag",
] = "1-4 months"
miso_data.loc[
    (miso_data["data_availability"] == "cems_and_eia")
    & (miso_data["reporting_frequency_code"].isna()),
    "reporting_lag",
] = "1-4 months"  # assume worse case
miso_data.loc[
    miso_data[
        [
            "net_generation_mwh",
            "fuel_consumed_for_electricity_mmbtu",
            "co2_mass_lb_for_electricity",
        ]
    ]
    .isnull()
    .all(axis=1),
    "reporting_lag",
] = "no_data_reported"
miso_data["reporting_lag"] = miso_data["reporting_lag"].astype("category")

miso_data

In [ ]:
# get statistics about where plants report
miso_data.groupby(
    ["reporting_lag", "emissions_category"], dropna=False
)["plant_id_eia"].count().reset_index().pivot(
    index=["reporting_lag"],
    columns=["emissions_category"],
    values="plant_id_eia",
)

In [ ]:
# get statistics about where plants report
miso_data.groupby(
    [ "reporting_lag", "fuel_category"], dropna=False
)["plant_id_eia"].count().reset_index().pivot(
    index=[ "reporting_lag"],
    columns=["fuel_category"],
    values="plant_id_eia",
)

In [ ]:
(miso_data.groupby(["emissions_category","reporting_lag"], dropna=True)[
    ["net_generation_mwh", "co2_mass_lb_for_electricity"]
].sum() ).round(1)

In [ ]:
plant_report = miso_data[
    [
        "plant_id_eia",
        "plant_name_eia",
        "state",
        "fuel_category",
        "emissions_category",
        "data_availability",
        "reporting_frequency_code",
        "reporting_lag",
        "net_generation_mwh",
        "co2_mass_lb_for_electricity",
    ]
]
plant_report = plant_report.rename(
    columns={"reporting_frequency_code": "eia923_reporting_frequency"}
)
#plant_report.to_csv(outputs_folder("miso_plant_reporting_analysis.csv"))

In [ ]:
# create a filtered version that only includes certain utilities
# utility_list = ['Entergy Arkansas Inc', "Ameren Illinois", 'Ameren Missouri']

utility_data = plant_data[
    plant_data["utility_name_eia"].str.contains("Ameren Illinois")
    | plant_data["transmission_distribution_owner_name"].str.contains("Ameren")
].copy()

# add a column for clean vs emitting
utility_data["emissions_category"] = "emitting"
utility_data.loc[
    utility_data["fuel_category"].isin(["hydro", "nuclear", "solar", "wind"]),
    "emissions_category",
] = "non-emitting"
utility_data.loc[
    utility_data["plant_primary_fuel"].isin(["MWH", "PUR", "WH"]),
    "emissions_category",
] = "non-emitting"

# change AM frequency to A
utility_data.loc[
    utility_data["reporting_frequency_code"] == "AM", "reporting_frequency_code"
] = "A"

utility_data.loc[
    utility_data["data_availability"] == "cems_only", "data_availability"
] = "cems_and_eia"
utility_data["data_availability"] = utility_data[
    "data_availability"
].cat.remove_unused_categories()


# add reporting lag column
utility_data["reporting_lag"] = "no_data_reported"
utility_data.loc[
    (utility_data["data_availability"] == "eia_only")
    & (utility_data["reporting_frequency_code"] == "A"),
    "reporting_lag",
] = "10-22 months"
utility_data.loc[
    (utility_data["data_availability"] == "eia_only")
    & (utility_data["reporting_frequency_code"] == "M"),
    "reporting_lag",
] = "2-3 months"
utility_data.loc[
    (utility_data["data_availability"] == "eia_only")
    & (utility_data["reporting_frequency_code"].isna()),
    "reporting_lag",
] = "10-22 months"  # assume worst case
utility_data.loc[
    (utility_data["data_availability"] == "cems_and_eia")
    & (utility_data["reporting_frequency_code"] == "M"),
    "reporting_lag",
] = "1-3 months"
utility_data.loc[
    (utility_data["data_availability"] == "cems_and_eia")
    & (utility_data["reporting_frequency_code"] == "A"),
    "reporting_lag",
] = "1-4 months"
utility_data.loc[
    (utility_data["data_availability"] == "cems_and_eia")
    & (utility_data["reporting_frequency_code"].isna()),
    "reporting_lag",
] = "1-4 months"  # assume worse case
utility_data.loc[
    utility_data[
        [
            "net_generation_mwh",
            "fuel_consumed_for_electricity_mmbtu",
            "co2_mass_lb_for_electricity",
        ]
    ]
    .isnull()
    .all(axis=1),
    "reporting_lag",
] = "no_data_reported"
utility_data["reporting_lag"] = utility_data["reporting_lag"].astype("category")

utility_data

In [ ]:
index_order = ["1-3 months", "2-3 months", "1-4 months", "10-22 months","no_data_reported"]

In [ ]:
# get statistics about where plants report
table = utility_data.groupby(
    ["reporting_lag", "emissions_category"], dropna=False
)["plant_id_eia"].count().reset_index().pivot(
    index=["reporting_lag"],
    columns=["emissions_category"],
    values="plant_id_eia",
)
table.loc[[i for i in index_order if i in table.index],:]

In [ ]:
# get statistics about where plants report
utility_data.groupby(
    [ "reporting_lag", "fuel_category"], dropna=False
)["plant_id_eia"].count().reset_index().pivot(
    index=[ "reporting_lag"],
    columns=["fuel_category"],
    values="plant_id_eia",
)

In [ ]:
table = (
    utility_data.groupby(["emissions_category", "reporting_lag"], dropna=True)[
        ["net_generation_mwh", "co2_mass_lb_for_electricity"]
    ].sum()
).round(1).reset_index().pivot(
    index="reporting_lag",
    columns=["emissions_category"],
    values=["net_generation_mwh", "co2_mass_lb_for_electricity"],
)
table.loc[[i for i in index_order if i in table.index],:]

In [ ]:
table = (
    utility_data.groupby(["emissions_category", "reporting_lag"], dropna=True)[
        ["net_generation_mwh", "co2_mass_lb_for_electricity"]
    ].sum() / utility_data[["net_generation_mwh","co2_mass_lb_for_electricity"]].sum() *100
).round(1).reset_index().pivot(
    index="reporting_lag",
    columns=["emissions_category"],
    values=["net_generation_mwh", "co2_mass_lb_for_electricity"],
)
table.loc[[i for i in index_order if i in table.index],:].sum()



## What percentage of the EIA data we actually use is from annually-reporting plants?
Even though ~10% of the data in EIA-923 is reported annually, we do not directly use all of the EIA-923 in our final results, since in some cases we have CEMS data that supercedes the EIA data. So what percentage of the EIA data we actually do use comes from annually-reporting plants?

In [ ]:
annual_eia_used = eia923_allocated[eia923_allocated["hourly_data_source"] != "cems"].groupby(["reporting_frequency_code"], dropna=False)[["fuel_consumed_mmbtu", "net_generation_mwh","co2_mass_lb"]].sum() / eia923_allocated[["fuel_consumed_mmbtu", "net_generation_mwh","co2_mass_lb"]].sum() * 100
annual_eia_used.loc["Total Percent"] = annual_eia_used.sum()
annual_eia_used

In [ ]:
annual_eia_used.loc["A",:].rename("% of output data from EIA annual reporters")

## What percentage of the data that we get from multiple sources is annually reported?
For subplants where all of the data comes from EIA, the annual reporting may lead to lower-quality allocation of the data to each month and hour, but there should not be a risk of double counting. However, for some plants, we draw data for some months from CEMS and other months from EIA-923. If the annually-reported data was not allocated to the correct month (relative to the CEMS data), there is a risk that we may double-count or under count data from that subplant. 

In [ ]:
multi_source_subplants = eia923_allocated[
    ["plant_id_eia", "subplant_id", "hourly_data_source"]
].drop_duplicates().drop(columns="hourly_data_source")

multi_source_subplants = multi_source_subplants[
            multi_source_subplants.duplicated(
                subset=["plant_id_eia", "subplant_id"])]

multi_source_subplants = eia923_allocated.merge(multi_source_subplants, how="inner", on=["plant_id_eia", "subplant_id"])


In [ ]:
# what percent of the total EIA-923 data comes from subplants with annually-reported data and multiple sources?
multi_source_summary = (multi_source_subplants.groupby(["respondent_frequency"], dropna=False)[["fuel_consumed_mmbtu", "net_generation_mwh","co2_mass_lb"]].sum() / eia923_allocated[["fuel_consumed_mmbtu", "net_generation_mwh","co2_mass_lb"]].sum() * 100)
multi_source_summary.loc["Total Percent"] = multi_source_summary.sum()
multi_source_summary

In [ ]:
multi_source_summary.loc["A",:].rename("% of output data mixing CEMS and annually-reported EIA data")

In [ ]:
pd.concat([pd.DataFrame(data_from_annual.loc["A",:].rename("% of EIA-923 input data from EIA annual reporters").round(2)).T, pd.DataFrame(annual_eia_used.loc["A",:].rename("% of output data from EIA annual reporters").round(2)).T, pd.DataFrame(multi_source_summary.loc["A",:].rename("% of output data mixing CEMS and annually-reported EIA data").round(2)).T], axis=0)